In [13]:
%matplotlib inline


Node Classification with DGL
============================




In [26]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [27]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [50]:
g = dataset[0]

In [29]:
writer = SummaryWriter()



A DGL graph can store node features and edge features in two
dictionary-like attributes called ``ndata`` and ``edata``.
In the DGL Cora dataset, the graph contains the following node features:

- ``train_mask``: A boolean tensor indicating whether the node is in the
  training set.

- ``val_mask``: A boolean tensor indicating whether the node is in the
  validation set.

- ``test_mask``: A boolean tensor indicating whether the node is in the
  test set.

- ``label``: The ground truth node category.

-  ``feat``: The node features.




In [30]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [65]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, 16)
        self.conv2 = GraphConv(16, num_classes)

    def forward(self, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
#features = torch.randn(g.number_of_nodes(), g.ndata["feat"].shape[1])
feature_dim = g.ndata['feat'].shape[1]
model = GCN(feature_dim, dataset.num_classes)

sample_features = torch.randn(g.number_of_nodes(), feature_dim)
tranced_model = torch.jit.trace(model, sample_features)

writer.add_graph(tranced_model, sample_features)


c:\Users\solma\anaconda3\Lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:407: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (graph.in_degrees() == 0).any():
c:\Users\solma\anaconda3\Lib\site-packages\dgl\heterograph.py:2926: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  if len(vid_tensor) > 0 and F.as_scalar(F.min(vid_tensor, 0)) < 0 < len(
c:\Users\solma\anaconda3\Lib\site-packages\dgl\backend\pytorch\tensor.py:57: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a 

TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%self.1 : __torch__.GCN,
		        %in_feat : Tensor):
		    %conv2 : __torch__.dgl.nn.pytorch.conv.graphconv.GraphConv = prim::GetAttr[name="conv2"](%self.1)
		    %conv1 : __torch__.dgl.nn.pytorch.conv.graphconv.GraphConv = prim::GetAttr[name="conv1"](%self.1)
		    %13 : int = prim::Constant[value=0](), scope: __module.conv1 # c:\Users\solma\anaconda3\Lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:433:0
		    %14 : float = prim::Constant[value=-0.5](), scope: __module.conv1 # c:\Users\solma\anaconda3\Lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:430:0
		    %15 : int = prim::Constant[value=1](), scope: __module.conv1 # c:\Users\solma\anaconda3\Lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:428:0
		    %16 : NoneType = prim::Constant(), scope: __module.conv1
		    %17 : bool = prim::Constant[value=0](), scope: __module.conv1 # c:\Users\solma\anaconda3\Lib\si

DGL provides implementation of many popular neighbor aggregation
modules. You can easily invoke them with one line of code.




Training the GCN
----------------

Training this GCN is similar to training other PyTorch neural networks.




In [44]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.947, val acc: 0.110 (best 0.110), test acc: 0.110 (best 0.110)
In epoch 5, loss: 1.907, val acc: 0.348 (best 0.424), test acc: 0.349 (best 0.451)
In epoch 10, loss: 1.836, val acc: 0.476 (best 0.476), test acc: 0.515 (best 0.515)
In epoch 15, loss: 1.741, val acc: 0.540 (best 0.540), test acc: 0.577 (best 0.577)
In epoch 20, loss: 1.619, val acc: 0.604 (best 0.604), test acc: 0.622 (best 0.622)
In epoch 25, loss: 1.474, val acc: 0.678 (best 0.678), test acc: 0.690 (best 0.690)
In epoch 30, loss: 1.310, val acc: 0.710 (best 0.710), test acc: 0.712 (best 0.712)
In epoch 35, loss: 1.134, val acc: 0.726 (best 0.726), test acc: 0.734 (best 0.734)
In epoch 40, loss: 0.956, val acc: 0.742 (best 0.742), test acc: 0.755 (best 0.749)
In epoch 45, loss: 0.787, val acc: 0.748 (best 0.750), test acc: 0.762 (best 0.758)
In epoch 50, loss: 0.637, val acc: 0.754 (best 0.754), test acc: 0.766 (best 0.769)
In epoch 55, loss: 0.509, val acc: 0.754 (best 0.756), test acc: 0.772 (best 0